In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/akashgaikwad746/MLOps_Wine_Quality_Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="akashgaikwad746"
os.environ["MLFLOW_TRACKING_PASSWORD"]="8213a7d6d852923d36fdd89244fe83cdd3eb73c7"

In [2]:
import os
%pwd

'c:\\Users\\akash\\MLOps_Wine_Quality_Prediction\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\akash\\MLOps_Wine_Quality_Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from src.Wine_Quality_Prediction.constants import *
from src.Wine_Quality_Prediction.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/akashgaikwad746/MLOps_Wine_Quality_Prediction.mlflow"


        )
        return model_evaluation_config


In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

[2025-09-17 19:26:56,471: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2025-09-17 19:26:56,471: INFO: utils: NumExpr defaulting to 8 threads.]


In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)   # use tracking URI, not registry
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log params + metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # ✅ Save model locally first
            local_model_path = "artifacts/model_evaluation/model.pkl"
            joblib.dump(model, local_model_path)

            # ✅ Log as plain artifact instead of mlflow.sklearn.log_model
            mlflow.log_artifact(local_model_path, artifact_path="model")


In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-09-17 19:26:58,673: INFO: common: yaml file: C:\Users\akash\MLOps_Wine_Quality_Prediction\config\config.yaml loaded successfully]
[2025-09-17 19:26:58,675: INFO: common: yaml file: C:\Users\akash\MLOps_Wine_Quality_Prediction\params.yaml loaded successfully]
[2025-09-17 19:26:58,678: INFO: common: yaml file: C:\Users\akash\MLOps_Wine_Quality_Prediction\schema.yaml loaded successfully]
[2025-09-17 19:26:58,679: INFO: common: Created directory at: artifacts]
[2025-09-17 19:26:58,681: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-09-17 19:26:59,454: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run luminous-horse-32 at: https://dagshub.com/akashgaikwad746/MLOps_Wine_Quality_Prediction.mlflow/#/experiments/0/runs/ceed059e99714ea5ab7554379735fada
🧪 View experiment at: https://dagshub.com/akashgaikwad746/MLOps_Wine_Quality_Prediction.mlflow/#/experiments/0
